# Plot

In [ ]:
!pip install plotnine pandas -q

In [1]:
import pandas as pd,os
import plotnine as p9
from os import listdir
from os.path import join
import csv
import matplotlib.pyplot as plt
import ast
import re
import altair as alt
from altair_saver import save
import askcomm

# better viewing of tweet text
pd.set_option('display.max_colwidth', 150)
import warnings
warnings.filterwarnings('ignore')
p9.options.figure_size=(11,8)

## Load data

In [2]:
df_all = pd.read_csv(
    '../synthesis/data.jobcensus.rc.synthesized.2020-12-3.csv'
)
# df_all = df_all[df_all.JobYear > 2010] #Filter out 2010
print(len(df_all))
df_all[:2]

4442


,JobTitle,JobYear,JobLink,JobType,JobField,JobSource,JobLink_norm
0,"Abilene Christian University (Abilene, TX)",2011,http://www.acu.edu/academics/provost/positions.html#cas | http://www.acu.edu/academics/provost/documents/Faculty_Application1.pdf,TT,Rhetoric/Composition,Wiki,http://www.acu.edu/academics/provost/positions.html#cas | http://www.acu.edu/academics/provost/documents/Faculty_Application1.pdf
1,Alfred State College (NY),2011,http://www.higheredjobs.com/faculty/details.cfm?JobCode=175465551,TT,Rhetoric/Composition,Wiki,http://www.higheredjobs.com/faculty/details.cfm?JobCode=175465551


In [3]:
len(df_all)

4442

## Create DateTime Year Column

In [4]:
df_all['Year'] = pd.to_datetime(df_all['JobYear'], format="%Y") 
df_all['Year'] = pd.DatetimeIndex(df_all['Year']).year
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4442 entries, 0 to 4441
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   JobTitle      4442 non-null   object
 1   JobYear       4442 non-null   int64 
 2   JobLink       4030 non-null   object
 3   JobType       4442 non-null   object
 4   JobField      4442 non-null   object
 5   JobSource     4442 non-null   object
 6   JobLink_norm  4030 non-null   object
 7   Year          4442 non-null   int64 
dtypes: int64(2), object(6)
memory usage: 277.8+ KB


## Import Field Alias Data

In [5]:
df_alias = pd.read_csv('../synthesis/data.jobcensus.rc.field_aliases.csv')
df_alias.sample(5)

,JobField,FieldAlias,Count
25,Modern British,Modern British,26
12,Community Colleges,Community Colleges,353
20,Ethnic Studies,Ethnic Studies,1
5,Anglophone,Anglophone,133
2,African American,African American,53


## Merge Fields with Corpus

In [6]:
df=df_all.merge(df_alias,on='JobField',how='inner')
df[df.JobField=='Community Colleges']

,JobTitle,JobYear,JobLink,JobType,JobField,JobSource,JobLink_norm,Year,FieldAlias,Count
3683,Aims Community College (CO) - Full-Time English / Humanities Instructor,2014,http://www.higheredjobs.com/faculty/details.cfm?JobCode=175861805&Title=English%2FHumanities%20Instructor,TT,Community Colleges,Wiki,http://www.higheredjobs.com/faculty/details.cfm?JobCode=175861805&Title=English%2FHumanities%20Instructor,2014,Community Colleges,353
3684,American River College (CA) - TT Asst. Professor of English (Writing),2014,https://chroniclevitae.com/jobs/0000817869-01#sthash.s4JbEajH.dpuf,TT,Community Colleges,Wiki,https://chroniclevitae.com/jobs/0000817869-01#sthash.s4JbEajH.dpuf,2014,Community Colleges,353
3685,Arizona Western College - Professor of English (Fall 2014),2014,mailto:human.resources@azwestern.edu | https://chroniclevitae.com/jobs/0000828212-01#sthash.BzQOVy2W.dpuf,TT,Community Colleges,Wiki,mailto:human.resources@azwestern.edu | https://chroniclevitae.com/jobs/0000828212-01#sthash.BzQOVy2W.dpuf,2014,Community Colleges,353
3686,Baltimore City Community College (MD) - Asst. Professor of English,2014,mailto:careers@bccc.edu | http://www.bccc.edu/cms/lib05/MD11000285/Centricity/Domain/64/Assistant%20Professor%20Englishb.pdf | http://www.highered...,TT,Community Colleges,Wiki,mailto:careers@bccc.edu | http://www.bccc.edu/cms/lib05/MD11000285/Centricity/Domain/64/Assistant%20Professor%20Englishb.pdf | http://www.highered...,2014,Community Colleges,353
3687,Bellevue College (WA) - Full-time Tenure Track Faculty in English (2 Positions) - PRELIMINARY INTERVIEW SCHEDULED,2014,http://www.bellevuecollege.edu/jobs | https://chroniclevitae.com/jobs/0000819336-01,TT,Community Colleges,Wiki,http://www.bellevuecollege.edu/jobs | https://chroniclevitae.com/jobs/0000819336-01,2014,Community Colleges,353
...,...,...,...,...,...,...,...,...,...,...
4030,South Puget Sound Community College (WA) - TT English Faculty (2 Positions),2018,https://www.higheredjobs.com/faculty/details.cfm?JobCode=176628082&Title=English%20Professor,TT,Community Colleges,Wiki,https://www.higheredjobs.com/faculty/details.cfm?JobCode=176628082&Title=English%20Professor,2018,Community Colleges,353
4031,Truckee Meadows Community College (NV) - TT English Instructor (2 Positions),2018,https://nshe.wd1.myworkdayjobs.com/en-US/TMCC-External/job/TMCC---Truckee-Meadows-Community-College---Dandini-Campus/Tenure-Track-English-Instruct...,TT,Community Colleges,Wiki,https://nshe.wd1.myworkdayjobs.com/en-US/TMCC-External/job/TMCC---Truckee-Meadows-Community-College---Dandini-Campus/Tenure-Track-English-Instruct...,2018,Community Colleges,353
4032,Valencia College (FL) - Professor of English (incl. TT - multiple positions),2018,http://valenciacollege.edu/faculty/forms/credentials/ | https://valenciacollege.csod.com/ats/careersite/JobDetails.aspx?id=5321&source=highered | ...,TT,Community Colleges,Wiki,http://valenciacollege.edu/faculty/forms/credentials/ | https://valenciacollege.csod.com/ats/careersite/JobDetails.aspx?id=5321&source=highered | ...,2018,Community Colleges,353
4033,"Hostos Community College, CUNY (NY) - TT Asst. Professor - English (Multiple Positions)",2019,https://cuny.jobs/bronx-ny/assistant-professor-english/C5B83F6F720449539264CEE2DA789C39/job/,TT,Community Colleges,Wiki,https://cuny.jobs/bronx-ny/assistant-professor-english/C5B83F6F720449539264CEE2DA789C39/job/,2019,Community Colleges,353


In [7]:
df['JobYear'] = df['JobYear'].apply(int)
df.sample(5)

,JobTitle,JobYear,JobLink,JobType,JobField,JobSource,JobLink_norm,Year,FieldAlias,Count
1698,University of Tampa (FL) - Assistant Professor of English (Academic Writing) - 1 yr. position (Non-TT) - SEARCH CANCELLED,2016,http://www.ut.edu/artsandletters/ | http://www.Click2Apply.net/t7mssqhyfr | https://www.higheredjobs.com/faculty/details.cfm?JobCode=176151697&Tit...,TT,Rhetoric/Composition,Wiki,http://www.ut.edu/artsandletters/ | http://www.Click2Apply.net/t7mssqhyfr | https://www.higheredjobs.com/faculty/details.cfm?JobCode=176151697&Tit...,2016,Rhetoric/Composition,2443
1603,West Virginia University (WV) - Teaching Assistant Professor - English Composition/Associate Undergraduate Writing Coordinator (Non-TT) - ADDITION...,2016,https://chroniclevitae.com/jobs/0000901594-01#sthash.QCFkYkO1.dpuf,TT,Rhetoric/Composition,Wiki,https://chroniclevitae.com/jobs/0000901594-01#sthash.QCFkYkO1.dpuf,2016,Rhetoric/Composition,2443
1901,Georgia Southern University (GA) - Lecturer in First-Year Writing,2017,http://class.georgiasouthern.edu/ | https://careers.insidehighered.com/job/1258691/lecturer-search-67435-college-of-liberal-arts-and-social-scienc...,Non-TT,Rhetoric/Composition,Wiki,http://class.georgiasouthern.edu/ | https://careers.insidehighered.com/job/1258691/lecturer-search-67435-college-of-liberal-arts-and-social-scienc...,2017,Rhetoric/Composition,2443
1192,Dean College (MA) - Assistant Professor of English (Spring 2015 or Fall 2015),2015,https://deancollege.trackingtalent.com/opportunities/assistant-professor-of-english-,TT,Rhetoric/Composition,Wiki,https://deancollege.trackingtalent.com/opportunities/assistant-professor-of-english-,2015,Rhetoric/Composition,2443
2918,Saint Leo University - South Hampton Roads (VA) - TT Instructor/Assistant/Associate Professor of English,2016,https://www.saintleo.jobs/postings/15577 | https://www.higheredjobs.com/faculty/details.cfm?JobCode=176192995&Title=Instructor%2FAssistant%2FAssoc...,TT,Generalist,Wiki,https://www.saintleo.jobs/postings/15577 | https://www.higheredjobs.com/faculty/details.cfm?JobCode=176192995&Title=Instructor%2FAssistant%2FAssoc...,2016,Generalist,816


In [8]:
# filter
dfm = df
dfm = dfm[dfm.FieldAlias!='--']
dfm.sample(5)

,JobTitle,JobYear,JobLink,JobType,JobField,JobSource,JobLink_norm,Year,FieldAlias,Count
2782,Georgia Southwestern State University (GA) - Full-Time Lecturer in English (Generalist) - PRELIMINARY INTERVIEW STAGE,2015,https://gsw.peopleadmin.com/postings/1128 | https://chroniclevitae.com/jobs/0000876268-01#sthash.5f8iMF3J.dpuf,Non-TT,Generalist,Wiki,https://gsw.peopleadmin.com/postings/1128 | https://chroniclevitae.com/jobs/0000876268-01#sthash.5f8iMF3J.dpuf,2015,Generalist,816
572,"New York University - Director, Expository Writing",2013,http://chronicle.com/jobs/0000776845-01/,TT,Rhetoric/Composition,Wiki,http://chronicle.com/jobs/0000776845-01/,2013,Rhetoric/Composition,2443
231,Boise State University - Assistant Professor - CAMPUS VISITS SCHEDULED,2012,http://english.boisestate.edu/faculty-positions/ | https://academicjobs.wikia.org/index.php?title=Rhetoric/Composition_2012#top,TT,Rhetoric/Composition,Wiki,http://english.boisestate.edu/faculty-positions/ | https://academicjobs.wikia.org/index.php?title=Rhetoric/Composition_2012#top,2012,Rhetoric/Composition,2443
1177,Bowdoin College (ME) - Director of Writing and Rhetoric [non-TT] - CAMPUS INTERVIEW SCHEDULED,2015,https://chroniclevitae.com/jobs/0000852301-01#sthash.fRoNGb9r.dpuf,TT,Rhetoric/Composition,Wiki,https://chroniclevitae.com/jobs/0000852301-01#sthash.fRoNGb9r.dpuf,2015,Rhetoric/Composition,2443
1353,West Virginia University (WV) - Open Field Endowed Chair of English,2015,https://chroniclevitae.com/jobs/0000841089-01#sthash.dDG26bfB.dpuf,TT,Rhetoric/Composition,Wiki,https://chroniclevitae.com/jobs/0000841089-01#sthash.dDG26bfB.dpuf,2015,Rhetoric/Composition,2443


## Plot 0: All jobs

In [9]:
df_count0 = dfm[dfm.JobType!='Postdoc'].groupby(['Year']).count().reset_index()
df_count0

,Year,JobTitle,JobYear,JobLink,JobType,JobField,JobSource,JobLink_norm,FieldAlias,Count
0,2011,311,311,265,311,311,311,265,311,311
1,2012,444,444,421,444,444,444,421,444,444
2,2013,422,422,361,422,422,422,361,422,422
3,2014,603,603,553,603,603,603,553,603,603
4,2015,655,655,641,655,655,655,641,655,655
5,2016,416,416,393,416,416,416,393,416,416
6,2017,382,382,371,382,382,382,371,382,382
7,2018,391,391,364,391,391,391,364,391,391
8,2019,277,277,260,277,277,277,260,277,277
9,2020,139,139,117,139,139,139,117,139,139


In [ ]:
fig=(
    p9.ggplot(p9.aes(x='Year',y='Count'),df_count0) 
    + p9.geom_line()
    + p9.geom_text(p9.aes(label='Count'),size=10)
    + p9.scale_x_continuous(breaks=range(2011,2021))
    + p9.ggtitle('Number of academic jobs (excl. postdocs), 2011-2020 (as of 12/1/2020)')
)
fig.save('figures/fig0.num_jobs.png',dpi=300)
fig

In [10]:
df_count0.set_index("Year")
df_count0

,Year,JobTitle,JobYear,JobLink,JobType,JobField,JobSource,JobLink_norm,FieldAlias,Count
0,2011,311,311,265,311,311,311,265,311,311
1,2012,444,444,421,444,444,444,421,444,444
2,2013,422,422,361,422,422,422,361,422,422
3,2014,603,603,553,603,603,603,553,603,603
4,2015,655,655,641,655,655,655,641,655,655
5,2016,416,416,393,416,416,416,393,416,416
6,2017,382,382,371,382,382,382,371,382,382
7,2018,391,391,364,391,391,391,364,391,391
8,2019,277,277,260,277,277,277,260,277,277
9,2020,139,139,117,139,139,139,117,139,139


In [23]:
domain = ['2011', '2012','2013','2014','2015','2016','2017','2018','2019','2020']
range_ = ['red', 'blue','goldenrod','cadetblue','orchid','firebrick','peru','slateblue','olivedrab','seagreen']

chart = alt.Chart(df_count0).mark_line(point=True, opacity=1).encode(
    x='Year:O',
    y='Count',
    tooltip=['Count'],
    text='Count:Q',
)

text = chart.mark_text(
    align='center',
    baseline='middle',
    dy=-10  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Count:Q'
)

# save(chart, '../assets/figures/rc-jobs-overall.svg')
# save(chart, '../assets/figures/rc-jobs-overall.png')

(chart+text).properties(height=500, width=800).interactive()

alt.LayerChart(...)

## Plot 1: By job type

In [25]:
df_count_job_type = dfm.groupby(['JobType','Year']).count().reset_index()
df_count_job_type

,JobType,Year,JobTitle,JobYear,JobLink,JobField,JobSource,JobLink_norm,FieldAlias,Count
0,Non-TT,2011,28,28,24,28,28,24,28,28
1,Non-TT,2012,40,40,39,40,40,39,40,40
2,Non-TT,2013,52,52,46,52,52,46,52,52
3,Non-TT,2014,65,65,59,65,65,59,65,65
4,Non-TT,2015,79,79,72,79,79,72,79,79
5,Non-TT,2016,72,72,65,72,72,65,72,72
6,Non-TT,2017,56,56,54,56,56,54,56,56
7,Non-TT,2018,61,61,57,61,61,57,61,61
8,Non-TT,2019,62,62,59,62,62,59,62,62
9,Non-TT,2020,26,26,20,26,26,20,26,26


In [26]:
chart = alt.Chart(df_count_job_type).mark_line(point=True).encode(
    x='Year:O',
    y='Count',
    strokeDash=alt.StrokeDash('JobType', legend=None),
    color=alt.Color('JobType', sort=['TT', 'Non-TT', 'Postdoc', 'Unknown']),
    tooltip=['Count'],
)

text = chart.mark_text(
    align='center',
    baseline='middle',
    dy=-10  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Count:Q'
)

# save(chart, '../assets/figures/rc-jobs-overall.svg')
# save(chart, '../assets/figures/rc-jobs-overall.png')

(chart+text).properties(height=500, width=800).interactive()

alt.LayerChart(...)

## Plot 2: By field

In [27]:
df_count_field = dfm.groupby(['Year','FieldAlias']).count().reset_index()
df_count_field

,Year,FieldAlias,JobTitle,JobYear,JobLink,JobType,JobField,JobSource,JobLink_norm,Count
0,2011,Generalist,66,66,62,66,66,66,62,66
1,2011,Rhetoric/Composition,214,214,191,214,214,214,191,214
2,2011,TESOL/Applied Linguistics,32,32,13,32,32,32,13,32
3,2012,Generalist,76,76,64,76,76,76,64,76
4,2012,Rhetoric/Composition,333,333,322,333,333,333,322,333
5,2012,TESOL/Applied Linguistics,37,37,37,37,37,37,37,37
6,2013,Generalist,63,63,59,63,63,63,59,63
7,2013,New Media and Digital Humanities,104,104,92,104,104,104,92,104
8,2013,Rhetoric/Composition,255,255,215,255,255,255,215,255
9,2013,TESOL/Applied Linguistics,16,16,10,16,16,16,10,16


In [28]:
fields = dfm.FieldAlias.value_counts().index
# top10 = dfm[dfm.JobType!='Postdoc'].FieldAlias.value_counts().iloc[:10].index
fields

Index(['Rhetoric/Composition', 'Generalist',
       'New Media and Digital Humanities', 'Community Colleges',
       'Humanities and Social Sciences Postdocs', 'TESOL/Applied Linguistics',
       'Communication and Media Studies'],
      dtype='object')

In [29]:
df_fields = df_count_field[df_count_field.FieldAlias.isin(fields)]
df_fields

,Year,FieldAlias,JobTitle,JobYear,JobLink,JobType,JobField,JobSource,JobLink_norm,Count
0,2011,Generalist,66,66,62,66,66,66,62,66
1,2011,Rhetoric/Composition,214,214,191,214,214,214,191,214
2,2011,TESOL/Applied Linguistics,32,32,13,32,32,32,13,32
3,2012,Generalist,76,76,64,76,76,76,64,76
4,2012,Rhetoric/Composition,333,333,322,333,333,333,322,333
5,2012,TESOL/Applied Linguistics,37,37,37,37,37,37,37,37
6,2013,Generalist,63,63,59,63,63,63,59,63
7,2013,New Media and Digital Humanities,104,104,92,104,104,104,92,104
8,2013,Rhetoric/Composition,255,255,215,255,255,255,215,255
9,2013,TESOL/Applied Linguistics,16,16,10,16,16,16,10,16


In [30]:
chart_field = alt.Chart(df_count_field).mark_line(point=True).encode(
    x='Year:O',
    y='Count',
    strokeDash=alt.StrokeDash('FieldAlias', legend=None),
    color=alt.Color('FieldAlias', sort=['Rhetoric/Composition', 'Generalist',
       'New Media and Digital Humanities', 'Community Colleges',
       'Humanities and Social Sciences Postdocs', 'TESOL/Applied Linguistics',
       'Communication and Media Studies']),
    tooltip=['Count'],
)

text = chart_field.mark_text(
    align='center',
    baseline='middle',
    dy=-10  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Count:Q'
)

# save(chart_field, '../assets/figures/rc-jobs-overall.svg')
# save(chart_field, '../assets/figures/rc-jobs-overall.png')

(chart_field+text).properties(height=500, width=800).interactive()

alt.LayerChart(...)

In [ ]:
df_count_field[df_count_field.Year == 2020]

## Plot 3: By field (all)

In [ ]:
df_count3=dfm[dfm.JobType!='Postdoc'].groupby(['JobYear','FieldAlias']).count().reset_index()
df_count3=df_count3[~df_count3.FieldAlias.str.contains('Unknown')]

# not consistently tracked by wiki:
df_count3=df_count3[~df_count3.FieldAlias.str.contains('Languages')] # not tracked by wiki data
df_count3=df_count3[~df_count3.FieldAlias.str.contains('Communication/Media Studies')] # not tracked by wiki data
df_count3=df_count3[~df_count3.FieldAlias.str.contains('Poetry')] # not tracked by wiki data

In [ ]:
# add zeros
unique_years = set(df_count3.JobYear)
missing_ld=[]
for field,fielddf in df_count3.groupby('FieldAlias'):
    field_years = set(fielddf.JobYear)
    for year in unique_years - field_years:
        dx=dict(fielddf.iloc[0])
        dx['JobYear']=year
        for k,v in dx.items():
            if not k in {'JobYear','FieldAlias'}:# and type(v)==int:
                dx[k]=0
        missing_ld.append(dx)
df_missing=pd.DataFrame(missing_ld)
df_count3=df_count3.append(df_missing)
df_count3

In [ ]:
fig=(
    p9.ggplot(p9.aes(x='JobYear',y='Count'), df_count3[df_count3.FieldAlias!='Postdoc'])
    + p9.geom_line()
    + p9.facet_wrap('FieldAlias')
    + p9.geom_text(p9.aes(label='Count'),size=8)
    + p9.ggtitle('Number of academic jobs per field (excl. postdocs), 2010-2020 (as of 11/24/2020)')
)
fig.save('figures/fig3.num_jobs_per_field_all.png',dpi=300)
fig

## Explore issues

### Unknown job types?

In [ ]:
df_unknown = dfm[(dfm.JobType=='Unknown')]
df_unknown.to_csv('../synthesis/data.unknown_jobtypes.csv',index=False)
df_unknown